<a href="https://colab.research.google.com/github/ulicezdravjainsamooskrbe-png/register-povrsin/blob/main/Generiranje_mape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import folium
import gspread
import os

# Authorize gspread using your service account credentials
gc = gspread.service_account(filename='credentials.json')

# List of your spreadsheet URLs or titles
spreadsheet_urls = [
    "https://docs.google.com/spreadsheets/d/1Zul-oksSymdDdcKc8KgHx4_ruIGtPzwp0r9z3-21DHE/edit",
    "https://docs.google.com/spreadsheets/d/1sfSGx9Tmu-t1SaHcM8w0vyou_p42ur2QYZywBPo_AqU/edit",
    "https://docs.google.com/spreadsheets/d/1zsIpk5qbVlgFY06GXRxlzmbgUEaeLDJhohvuX6oHUZs/edit"
]

colors = ['blue', 'red', 'green', 'orange', 'purple', 'darkred', 'lightred', 'beige']
icons = ['info', 'star', 'leaf', 'map-marker', 'camera', 'home', 'cloud', 'heart']


# Create a base map
map_center = [46.167, 14.305]
my_map = folium.Map(location=map_center,
    zoom_start=12,
    tiles='https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
    attr='Google Maps')

# Add the Google Satellite layer with a custom name
google_satellite = folium.TileLayer(
    tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google',
    name='Google Satellite' # This name will show up in the layer control
).add_to(my_map)

# Add the Google Hybrid layer as another option
google_hybrid = folium.TileLayer(
    tiles='https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
    attr='Google',
    name='Google Hybrid',
    retina=True
).add_to(my_map)

# Process each spreadsheet
for i, url in enumerate(spreadsheet_urls):

    # Get the color for the current spreadsheet
    # The modulo operator (%) ensures we don't go out of bounds if there are more URLs than colors
    marker_color = colors[i % len(colors)]
    icon = icons[i % len(icons)]

    try:
        # Open the spreadsheet by URL
        sh = gc.open_by_url(url)

        # Extract the title from the spreadsheet object
        spreadsheet_name = sh.title

        # Create a FeatureGroup with the spreadsheet's actual title as the name
        feature_group = folium.FeatureGroup(name=spreadsheet_name).add_to(my_map)

        worksheet = sh.get_worksheet(0) # Assumes your data is on the first sheet (index 0)

        # Get all records as a list of dictionaries
        data = worksheet.get_all_records()
        df = pd.DataFrame(data)

        # Add a marker for each row in the DataFrame
        for index, row in df.iterrows():
            lat = row.get('x')
            lon = row.get('y')
            title = row.get('title', 'No Title')
            comment = row.get('comment', 'No Comment')
            image_url = row.get('imageUrl', None)

            if pd.isna(lat) or pd.isna(lon):
                continue

            html_content = f"<h3>{title}</h3><p>{comment}</p>"
            if pd.notna(image_url):
                html_content += f'<img src="{image_url}" width="250">'

            folium.Marker(
                location=[lat, lon],
                popup=folium.Popup(html_content, max_width=300),
                tooltip=title,
                icon=folium.Icon(color=marker_color, icon=icon, prefix='fa')
            ).add_to(feature_group)

    except gspread.exceptions.SpreadsheetNotFound:
        print(f"Spreadsheet not found at URL: {url}. Please ensure the URL is correct and shared with the service account.")
        continue
    except Exception as e:
        print(f"An error occurred: {e}")

# Add the layer control to the map
folium.LayerControl().add_to(my_map)

# Save the map to an HTML file
output_file = "index.html"
my_map.save(output_file)

print(f"Map created successfully! Open '{output_file}' in your web browser.")

Map created successfully! Open 'index.html' in your web browser.
